In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from src.utils.config import config
import warnings
#np.random.seed(34)
warnings.filterwarnings('ignore')

# 1. Feature Engineering
Original features: 3 settings + 21 sensors = 24 features

In [ ]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i+1) for i in range(0,21)]

In [ ]:
prepared_folder = config.PREPARED_DATA_PATH

train_df = pd.read_csv(prepared_folder / "train-all-prepared.csv", index_col=False)
test_df = pd.read_csv(prepared_folder / "test-all-prepared.csv", index_col=False)

In [ ]:
train_df

## 1. Rolling Window Features
Moving averages of sensor readings over last N cycles
sensor_1_rolling_mean_5 = average of sensor 1 over last 5 cycles
Smooths out noise, reveals underlying trends

In [ ]:
def create_rolling_features(df, window_sizes=[3, 5, 10]):
    df_result = df.copy()

    df_result = df_result.sort_values(['subset', 'unit_number', 'time_cycles'])

    # Define feature columns (sensor readings)
    sensor_cols = [col for col in df_result.columns if col.startswith('s_')]
    settings_cols = [col for col in df_result.columns if col.startswith('setting_')]

    print(f"Creating rolling features for {len(sensor_cols)} sensor and {len(settings_cols)} settings columns...")

    for window_size in window_sizes:
        print(f"  Processing window size {window_size}...")

        for col in sensor_cols + settings_cols:
            # Use transform to maintain original index alignment
            feature_name = f"{col}_roll_{window_size}"
            df_result[feature_name] = (
                df_result.groupby('unit_number')[col]
                .rolling(window=window_size, min_periods=1)
                .mean()
                .reset_index(level=0, drop=True)  # Remove the groupby level from MultiIndex
            )

    return df_result

In [ ]:
print("Creating rolling features...")
window_sizes = [3, 5, 10, 20]
#window_sizes = [3, 5, 10]
#window_sizes = [2, 3, 5]
#window_sizes = [2]
train_df = create_rolling_features(train_df, window_sizes=window_sizes)
test_df = create_rolling_features(test_df, window_sizes=window_sizes)

In [ ]:
rolling_cols = [c for c in train_df.columns if '_roll_' in c]
correlations = train_df[rolling_cols + ['RUL']].corr(method='spearman')['RUL'].abs().sort_values(ascending=False)

print("Top 10 rolling features correlated with RUL:")
print(correlations.head(11)[1:])  # Skip RUL itself

# Check settings rolling features specifically
setting_rolling = [c for c in rolling_cols if c.startswith('setting')]
print(f"\nSettings rolling features correlation with RUL:")
for col in setting_rolling:
    corr = train_df[col].corr(train_df['RUL'], method='spearman')
    print(f"{col}: {corr:.4f}")

In [ ]:
train_df

## 2. Delta/Rate Features
How much each sensor changed from previous cycle
Example: sensor_1_delta = current_value - previous_value
Captures degradation speed/acceleration

In [ ]:
def create_delta_features(df):
    df_sorted = df.sort_values(['subset', 'unit_number', 'time_cycles']).copy()

    feature_cols = setting_names + sensor_names
    print(f"Creating delta features for all {len(feature_cols)} columns")

    # Calculate deltas within each unit
    grouped = df_sorted.groupby('unit_number')
    for col in feature_cols:
        delta_name = f"{col}_delta"
        df_sorted[delta_name] = grouped[col].diff().fillna(0)

    delta_cols = len([c for c in df_sorted.columns if '_delta' in c])
    print(f"Created {delta_cols} delta features")
    return df_sorted

In [ ]:
print("Creating delta features...")
train_df = create_delta_features(train_df)
test_df = create_delta_features(test_df)

In [ ]:
# Let correlation with RUL tell us which deltas are useful
delta_cols = [c for c in train_df.columns if '_delta' in c]
delta_correlations = train_df[delta_cols + ['RUL']].corr(method='spearman')['RUL'].abs().sort_values(ascending=False)

print("Top 10 delta features correlated with RUL:")
print(delta_correlations.head(10))

## 3. Time-based Features
What: Normalized cycle position (0 to 1 across engine's life)
Example: cycle_norm = current_cycle / max_cycles_for_this_engine

❗❗❗ Won't do - skip this FE ❗❗❗
Train has data until faillure
Test has data until a certain point, and need to predict RUL from this point (cycle)
They cannot be normalized the same way, this will create issue in prediction

## 4. Interaction Features (Medium complexity)
What: Combine operating settings to capture complex conditions
Example: setting_1_x_setting_2 = setting_1 * setting_2
Why important: Equipment might behave differently under combined stress
Computation: Simple multiplication

Operating settings with each other (setting_1 × setting_2 × setting_3)
Settings with key sensors (high temperature + high pressure scenarios)
Physically related sensors (temperature sensors with pressure sensors)

`X` : captures "amplification" effects
`+` : captures "combined stress" effects
`Ratio/Division`: captures "efficiency" or normalized response

### 4.1 Settings × Settings: 3 features (systematic)

In [ ]:
def create_settings_x_settings_interaction_features(df):
    df_copy = df.copy()
    df_copy['setting_1_x_setting_2'] = df_copy['setting_1'] * df_copy['setting_2']
    df_copy['setting_1_x_setting_3'] = df_copy['setting_1'] * df_copy['setting_3']
    df_copy['setting_2_x_setting_3'] = df_copy['setting_2'] * df_copy['setting_3']
    interaction_cols = [c for c in df_copy.columns if c not in df.columns]
    print(f"Created {len(interaction_cols)} interaction for settings features")

    return df_copy, interaction_cols

In [ ]:
print("Creating settings x settings features...")
train_df, settings_interaction_cols = create_settings_x_settings_interaction_features(train_df)
test_df, _ = create_settings_x_settings_interaction_features(test_df)

In [ ]:
# Check correlations with RUL
settings_interaction_corr = train_df[settings_interaction_cols + ['RUL']].corr(method='spearman')['RUL'].abs().sort_values(ascending=False)

print("Settings×settings interactions correlated with RUL:")
print(settings_interaction_corr.head(4))

### 4.2 Settings × Sensors: 63 features (systematic)

In [ ]:
def create_settings_sensor_interactions(df):
    df_copy = df.copy()

    interaction_count = 0

    # Each setting × each sensor
    for setting in setting_names:
        for sensor in sensor_names:
            feature_name = f"{setting}_x_{sensor}"
            df_copy[feature_name] = df_copy[setting] * df_copy[sensor]
            interaction_count += 1

    print(f"Created {interaction_count} settings×sensors interaction features")
    print(f"({len(setting_names)} settings × {len(sensor_names)} sensors)")

    return df_copy


In [ ]:
print("Creating settings×sensors interaction features...")
train_df = create_settings_sensor_interactions(train_df)
test_df = create_settings_sensor_interactions(test_df)

In [ ]:
# Check correlations with RUL
settings_sensor_cols = [c for c in train_df.columns if c.startswith('setting_') and '_x_s_' in c]
settings_sensor_corr = train_df[settings_sensor_cols + ['RUL']].corr(method='spearman')['RUL'].abs().sort_values(ascending=False)

print("Top 10 settings×sensors interactions correlated with RUL:")
print(settings_sensor_corr.head(10))

### 4.3 Sensors x sensors:
Instead of all 210 possible combinations, let's group them by what they measure.

##### 4.3.1 Temperature Group

In [ ]:
def create_temperature_group_interaction_features(df):
    df_copy = df.copy()

    df_copy['temp_fan_to_lpc'] = df_copy['s_1'] * df_copy['s_2']           # Fan inlet × LPC outlet
    df_copy['temp_compression_ratio'] = df_copy['s_3'] / df_copy['s_2']    # HPC outlet / LPC outlet
    df_copy['temp_expansion_ratio'] = df_copy['s_4'] / df_copy['s_3']      # LPT outlet / HPC outlet
    df_copy['temp_overall_rise'] = df_copy['s_3'] - df_copy['s_1']         # HPC outlet - Fan inlet

    interaction_cols = [c for c in df_copy.columns if c not in df.columns]
    print(f"Created {len(interaction_cols)} interaction for temperature features")

    return df_copy, interaction_cols

In [ ]:
print("Creating temperature features...")
train_df, interaction_cols = create_temperature_group_interaction_features(train_df)
test_df, _ = create_temperature_group_interaction_features(test_df)

In [ ]:
# Check correlations with RUL
settings_interaction_corr = train_df[interaction_cols + ['RUL']].corr(method='spearman')['RUL'].abs().sort_values(ascending=False)

print("Settings×settings interactions correlated with RUL:")
print(settings_interaction_corr.head(10))

##### 4.3.2 Pressure Group

In [ ]:
def create_pressure_group_interaction_features(df):
    df_copy = df.copy()

    df_copy['pressure_ratio_fan'] = df_copy['s_7'] / (df_copy['s_5'] + 1e-6)         # HPC outlet / Fan inlet
    df_copy['pressure_bypass_vs_core'] = df_copy['s_6'] / (df_copy['s_7'] + 1e-6)    # Bypass / HPC outlet
    df_copy['pressure_drop_turbine'] = df_copy['s_7'] - df_copy['s_11']              # HPC outlet - HPC static

    interaction_cols = [c for c in df_copy.columns if c not in df.columns]
    print(f"Created {len(interaction_cols)} interaction for pressure features")

    return df_copy, interaction_cols

In [ ]:
print("Creating pressure group interaction features...")
train_df, pressure_interaction_cols = create_pressure_group_interaction_features(train_df)
test_df, _ = create_pressure_group_interaction_features(test_df)

In [ ]:
# Check correlations with RUL
pressure_interaction_corr = train_df[pressure_interaction_cols + ['RUL']].corr(method='spearman')['RUL'].abs().sort_values(ascending=False)

print("Pressure group interactions correlated with RUL:")
print(pressure_interaction_corr.head(10))

##### 4.3.3 Speed Group

In [ ]:
# Speed Group Interaction Features
def create_speed_group_interaction_features(df):
    df_copy = df.copy()

    df_copy['speed_fan_core_ratio'] = df_copy['s_8'] / (df_copy['s_9'] + 1e-6)       # Physical fan / Physical core
    df_copy['speed_corrected_ratio'] = df_copy['s_13'] / (df_copy['s_14'] + 1e-6)    # Corrected fan / Corrected core
    df_copy['speed_efficiency'] = df_copy['s_8'] / (df_copy['s_18'] + 1e-6)          # Physical / Required fan speed

    interaction_cols = [c for c in df_copy.columns if c not in df.columns]
    print(f"Created {len(interaction_cols)} interaction for speed features")

    return df_copy, interaction_cols

In [ ]:
print("Creating speed group interaction features...")
train_df, speed_interaction_cols = create_speed_group_interaction_features(train_df)
test_df, _ = create_speed_group_interaction_features(test_df)

In [ ]:
# Check correlations with RUL
speed_interaction_corr = train_df[speed_interaction_cols + ['RUL']].corr(method='spearman')['RUL'].abs().sort_values(ascending=False)

print("Speed group interactions correlated with RUL:")
print(speed_interaction_corr.head(10))

##### 4.3.4 Fuel & Air Group

In [ ]:
# Fuel & Air Group Interaction Features
def create_fuel_air_group_interaction_features(df):
    df_copy = df.copy()

    df_copy['fuel_air_efficiency'] = df_copy['s_12'] * df_copy['s_16']                          # Fuel flow ratio × Burner fuel-air
    df_copy['cooling_air_total'] = df_copy['s_20'] + df_copy['s_21']                            # HP + LP turbine cooling air
    df_copy['fuel_to_cooling'] = df_copy['s_12'] / (df_copy['s_20'] + df_copy['s_21'] + 1e-6)   # Fuel efficiency vs cooling

    interaction_cols = [c for c in df_copy.columns if c not in df.columns]
    print(f"Created {len(interaction_cols)} interaction for fuel & air features")

    return df_copy, interaction_cols

In [ ]:
print("Creating fuel & air group interaction features...")
train_df, fuel_air_interaction_cols = create_fuel_air_group_interaction_features(train_df)
test_df, _ = create_fuel_air_group_interaction_features(test_df)

In [ ]:
# Check correlations with RUL
fuel_air_interaction_corr = train_df[fuel_air_interaction_cols + ['RUL']].corr(method='spearman')['RUL'].abs().sort_values(ascending=False)

print("Fuel & air group interactions correlated with RUL:")
print(fuel_air_interaction_corr.head(10))

##### 4.3.5 Cross-System Group (2 interactions)

In [ ]:
# Cross-System Group Interaction Features
def create_cross_system_group_interaction_features(df):
    df_copy = df.copy()

    df_copy['thermal_pressure_stress'] = df_copy['s_3'] * df_copy['s_7']    # HPC temp × HPC pressure (most stressed point)
    df_copy['engine_load_indicator'] = df_copy['s_10'] * df_copy['s_15']    # Engine pressure ratio × Bypass ratio

    interaction_cols = [c for c in df_copy.columns if c not in df.columns]
    print(f"Created {len(interaction_cols)} interaction for cross-system features")

    return df_copy, interaction_cols

In [ ]:
print("Creating cross-system group interaction features...")
train_df, cross_system_interaction_cols = create_cross_system_group_interaction_features(train_df)
test_df, _ = create_cross_system_group_interaction_features(test_df)

In [ ]:
# Check correlations with RUL
cross_system_interaction_corr = train_df[cross_system_interaction_cols + ['RUL']].corr(method='spearman')['RUL'].abs().sort_values(ascending=False)

print("Cross-system group interactions correlated with RUL:")
print(cross_system_interaction_corr.head(10))

_____

# Feature Selection

In [ ]:
train_df.shape

In [ ]:
test_df.shape

Planning to use Random Forest, should be able to handle ~200 features

In [ ]:
train_df.columns

## 1. Random Forest Feature Importance

In [ ]:
# Prepare features and target
#train_to_use = train_df[train_df['subset'] == 1]
#test_to_use = test_df[test_df['subset'] == 1]

train_to_use = train_df
for subset_id in [1, 2, 3, 4]:
    mask = train_to_use['subset'] == subset_id
    train_to_use.loc[mask, 'unit_number'] += 1000 * subset_id

test_to_use = test_df
for subset_id in [1, 2, 3, 4]:
    mask = test_to_use['subset'] == subset_id
    test_to_use.loc[mask, 'unit_number'] += 1000 * subset_id

In [ ]:
rul_thresholds = {
    1: {'max': 145, 'min': 6},
    2: {'max': 194, 'min': 6},
    3: {'max': 145, 'min': 6},
    4: {'max': 194, 'min': 6}
}

# Apply different RUL filtering for each subset
filtered_dfs = []
for subset_id in [1, 2, 3, 4]:
    subset_data = train_to_use[train_to_use['subset'] == subset_id]
    max_rul = rul_thresholds[subset_id]['max']
    min_rul = rul_thresholds[subset_id]['min']

    filtered_subset = subset_data[
        (subset_data['RUL'] <= max_rul) &
        (subset_data['RUL'] >= min_rul)
    ]
    filtered_dfs.append(filtered_subset)

# Combine all filtered subsets back together
train_to_use = pd.concat(filtered_dfs, ignore_index=True)

In [ ]:
# Get unique unit numbers for splitting
unique_units = train_to_use['unit_number'].unique()
train_units, valid_units = train_test_split(unique_units, test_size=0.1, random_state=45)

# Split data based on unit numbers
train_mask = train_to_use['unit_number'].isin(train_units)
test_mask = train_to_use['unit_number'].isin(valid_units)

X_train = train_to_use[train_mask].drop(['unit_number', 'time_cycles', 'RUL'], axis=1)
y_train = train_to_use[train_mask]['RUL']

X_valid = train_to_use[test_mask].drop(['unit_number', 'time_cycles', 'RUL'], axis=1)
y_valid = train_to_use[test_mask]['RUL']

In [ ]:
# Scale features (keep subset as is since it's categorical)
scaler = MinMaxScaler()
feature_cols = [col for col in X_train.columns if col != 'subset']

X_train_scaled = X_train.copy()
X_valid_scaled = X_valid.copy()

X_train_scaled[feature_cols] = scaler.fit_transform(X_train[feature_cols])
X_valid_scaled[feature_cols] = scaler.transform(X_valid[feature_cols])


In [ ]:
rf = RandomForestRegressor(
    n_estimators=500,           # More trees for complex patterns
    max_depth=None,             # Let trees grow deep
    min_samples_split=5,        # Prevent overfitting
    min_samples_leaf=2,         # Balance bias-variance
    max_features='sqrt',       # Features per tree: √200 ~ 14 features per tree
    random_state=46,
    n_jobs=-1                   # XGBoost for GPU
)
rf.fit(X_train_scaled, y_train)
top_n_features = X_train_scaled.columns

In [ ]:
# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

# Display top 20 features
print("Top 20 Most Important Features:")
print(feature_importance.head(20))

In [ ]:
# Plot top 15 features
plt.figure(figsize=(10, 6))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance')
plt.title('Top 15 Feature Importances')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

top_50_features = feature_importance.head(50)['feature'].tolist()
print(f"\nSelected {len(top_50_features)} features")
print(f"Baseline performance - RMSE: {np.sqrt(((rf.predict(X_valid_scaled) - y_valid)**2).mean()):.2f}")

-> Iterate here to find best hyperparameter (and feature engineering - rolling windows sizes, etc) -> script, optuna, MLFlow
max_train_RUL_to_use, etc

## Full train with feature selection

In [ ]:
n = 300
top_n_features = feature_importance['feature'].iloc[-n:]  # Last N feature names
top_n_features

In [ ]:
#X_train = train_to_use[top_n_features]
X_train = train_to_use.drop(['unit_number', 'time_cycles', 'RUL'], axis=1)
y_train = train_to_use['RUL']

scaler = MinMaxScaler()
feature_cols = [col for col in X_train.columns if col != 'subset']

X_train_scaled = X_train.copy()
X_train_scaled[feature_cols] = scaler.fit_transform(X_train[feature_cols])


rf = RandomForestRegressor(
    n_estimators=rf.n_estimators,               # More trees for complex patterns
    max_depth=rf.max_depth,                     # Let trees grow deep
    min_samples_split=rf.min_samples_split,     # Prevent overfitting
    min_samples_leaf=rf.min_samples_leaf,       # Balance bias-variance
    max_features=rf.max_features,               # Features per tree: √200 ~ 14 features per tree
    random_state=rf.random_state,
    n_jobs=-1                                   # XGBoost for GPU
)
rf.fit(X_train_scaled, y_train)

## Predict on Test

In [ ]:
test_summary = test_to_use.groupby('unit_number').agg({
    'time_cycles': 'max',
    'RUL': 'min'
}).reset_index()

In [ ]:
# Prepare test data - only last row per engine unit for RUL prediction
# First, let's verify that the last row (max time_cycles) is what we want to predict on
test_summary = test_to_use.groupby('unit_number').agg({
    'time_cycles': 'max',
    'RUL': 'min'
}).reset_index()

# Assert that for each unit, there's only one row with max time_cycles
for unit in test_to_use['unit_number'].unique():
    unit_data = test_to_use[test_to_use['unit_number'] == unit]
    max_time_rows = unit_data[unit_data['time_cycles'] == unit_data['time_cycles'].max()]
    assert len(max_time_rows) == 1, f"Unit {unit}: Multiple rows with same max time_cycles"

print("✓ Verified: Each engine unit has exactly one row with maximum time_cycles")

# Get only the last row (highest time_cycles) for each engine unit
test_last_rows = test_to_use.loc[test_to_use.groupby('unit_number')['time_cycles'].idxmax()]

print(f"Original test data shape: {test_to_use.shape}")
print(f"Test data (last rows only) shape: {test_last_rows.shape}")

#X_test = test_last_rows[top_n_features]
X_test = test_last_rows.drop(['unit_number', 'time_cycles', 'RUL'], axis=1)
y_test = test_last_rows['RUL']

X_test_scaled = X_test.copy()
X_test_scaled[feature_cols] = scaler.transform(X_test[feature_cols])


# Make predictions
y_pred = rf.predict(X_test_scaled)

# Calculate RMSE
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Test RMSE: {rmse:.2f}")

# Optional: Additional metrics
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MAE: {mae:.2f}")
print(f"Test R²: {r2:.3f}")

# Optional: Prediction vs Actual plot
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual RUL')
plt.ylabel('Predicted RUL')
plt.title(f'Predictions vs Actual (RMSE: {rmse:.2f})')
plt.tight_layout()
plt.show()

In [ ]:
# More trees: n_estimators=500
# Tune hyperparameters: max_depth, min_samples_split
# Keep top N features only: top_30_features = feature_importance.head(30)['feature'].tolist()

## 2. Mutual Information

In [ ]:
# from sklearn.feature_selection import mutual_info_regression
#
# mi_scores = mutual_info_regression(X, y)
# mi_df = pd.DataFrame({
#     'feature': X.columns,
#     'mutual_info': mi_scores
# }).sort_values('mutual_info', ascending=False)

## 3. Recursive Feature Elimination (RFE)

In [ ]:
# from sklearn.feature_selection import RFE
#
# rfe = RFE(estimator=RandomForestRegressor(n_estimators=100),
#           n_features_to_select=50)
# rfe.fit(X, y)
# selected_features = X.columns[rfe.support_]

## 4. Variance Threshold

In [ ]:
# from sklearn.feature_selection import VarianceThreshold
#
# # Remove features with very low variance
# selector = VarianceThreshold(threshold=0.01)
# X_filtered = selector.fit_transform(X)

_____

# Cycle selection - if late submit for competition

Idea for later: Since we pre-compute time feature, maybe we could only train on cycle that are close to RUL.
Need to analyse RUL distribution of test.
Need to make sure we only receive last cycle to predict for competition